In [1]:
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from pickle import dump
from IPython.display import display
import seaborn as sns
import tensorflow 
from tensorflow import keras
from keras import backend as K
tensorflow.random.set_seed(1)

In [2]:
dara_path='./dataset/'
def create_dataframe(data_path):
    df = []
    for c in ['@','&','#','$','0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','I','J','K','L','M','N','P','Q','R','S','T','U','V','W','X','Y','Z']:
        class_folder = os.path.join(data_path, c)
        for f in os.listdir(class_folder):
            f_path = os.path.join(class_folder, f)
            if f_path.endswith('.jpg'):
                df.append([f_path, c])
    df = pd.DataFrame(df, columns=('filename', 'class'))
    
    return df

In [3]:
IMG_DIM=224
DATA_PATH = './dataset/'
CLASSES = sorted(['@','&','#','$','0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','I','J','K','L','M','N','P','Q','R','S','T','U','V','W','X','Y','Z'])
NUM_CLASSES = len(CLASSES)
print(CLASSES)

['#', '$', '&', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [4]:
df = create_dataframe(os.path.join(DATA_PATH, 'Train'))
df_test = create_dataframe(os.path.join(DATA_PATH, 'Validation'))

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
df_train, df_val = train_test_split(df, test_size=0.30, random_state=0)

In [8]:
print(sorted(df_val['class'].unique()))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
train_gen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=10,
    zoom_range = 0.1,  
    width_shift_range=0.1, 
    height_shift_range=0.1,     
    fill_mode='nearest'
        ).flow_from_dataframe(
    df_train,
    target_size=(IMG_DIM, IMG_DIM),
    classes=CLASSES,
    batch_size=32,
    class_mode='sparse'
)

val_gen = ImageDataGenerator(rescale=1/255).flow_from_dataframe(
    df_val,
    target_size=(IMG_DIM, IMG_DIM),
    classes=CLASSES,
    batch_size=32,
    class_mode='sparse'
)

test_gen = ImageDataGenerator(rescale=1/255).flow_from_dataframe(
    df_test,
    target_size=(IMG_DIM, IMG_DIM),
    classes=CLASSES,
    shuffle=False,
    batch_size=32,
    class_mode='sparse'
)

Found 525859 validated image filenames belonging to 39 classes.
Found 225369 validated image filenames belonging to 39 classes.
Found 18104 validated image filenames belonging to 39 classes.


In [16]:
def create_model_vgg16():
    vgg16_model=tensorflow.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(IMG_DIM, IMG_DIM, 3))
    model = tensorflow.keras.Sequential()
    for layer in vgg16_model.layers[:-1]:
        model.add(layer)
    for layer in model.layers:
        layer.trainable=False
    model.add(keras.layers.Dense(NUM_CLASSES, activation='softmax'))
    model.compile(loss='sparse_ategorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [11]:
from tensorflow.keras.applications.resnet import ResNet152
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

In [12]:
def create_model_resnet():

    pretrained = ResNet152(
        input_shape=(IMG_DIM, IMG_DIM,3),
        include_top=False
    )
    pretrained.trainable = False

    model = Sequential()
    model.add(pretrained)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [13]:
model_weights_file_path = "resnet.hdf5"

In [17]:
from tensorflow.keras.layers import Flatten
from keras.models import Sequential

model = create_model_resnet()

234698864/234698864 [==============================] - 303s 1us/step

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 39)                79911     
                                                                 
Total params: 58450855 (222.97 MB)
Trainable params: 79911 (312.15 KB)
Non-trainable params: 58370944 (222.67 MB)
_________________________________________________________________


In [18]:
model.input_shape

(None, 224, 224, 3)

In [21]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


checkpoint = ModelCheckpoint(filepath=model_weights_file_path, monitor="val_loss", verbose=1, save_best_only=True, mode="min", save_weights_only=True)
es = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=2)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1, mode='min')
num_epochs = 10  # Define the number of epochs

history = model.fit(train_gen,
                    epochs=num_epochs,      
                    
                    validation_steps=150,
                    validation_data=val_gen,
                    callbacks=[checkpoint, es, lr_reduce], 
                    batch_size=32,
                    verbose=1)

Epoch 1/10
   15/16434 [..............................] - ETA: 16:11:52 - loss: 3.0279 - accuracy: 0.1854

KeyboardInterrupt: 

In [ ]:
def plot_loss_accuracy(history):
    plt.figure(figsize=[8,6])
    plt.plot(history.history['loss'],'r',linewidth=3.0)
    plt.plot(history.history['val_loss'],'b',linewidth=3.0)
    plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
    plt.xlabel('Epochs ',fontsize=16)
    plt.ylabel('Loss',fontsize=16)
    plt.title('Loss Curves',fontsize=16)

    plt.figure(figsize=[8,6])
    plt.plot(history.history['accuracy'],'g',linewidth=3.0)
    plt.plot(history.history['val_accuracy'],'b',linewidth=3.0)
    plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
    plt.xlabel('Epochs ',fontsize=16)
    plt.ylabel('Accuracy',fontsize=16)
    plt.title('Accuracy Curves',fontsize=16)

In [ ]:
plot_loss_accuracy(history)